In [1]:
from elasticsearch import Elasticsearch
from algorithm import index
import pandas as pd

In [2]:
data_path = "./data/csl_camera_readly.tsv"
with open(data_path, "r+", encoding="utf-8") as file:
    file.seek(0)
    file.write(file.read().replace('"', '`'))


In [3]:
data = pd.read_table(
    data_path,
    on_bad_lines="skip",
    header=None,
)
data = data.rename(
    columns={
        i: k for i, k in enumerate([
            "title", "content", "keywords", "subject", "category"
        ])}
)
data

,title,content,keywords,subject,category
0,谷物联合收获机自动测产系统设计-基于变权分层激活扩散模型,为了使联合收割机具有自动测产功能，提出了一种基于变权分层激活扩散的产量预测误差剔除模型，并使...,联合收割机_测产系统_变权分层_激活扩散,农业工程,工学
1,酞菁改性聚苯乙炔高分子的微波介电性能研究,"针对电磁环境的严重污染,克服目前电磁屏蔽的弊端,用吸波材料从根本上消除电磁污染是关注的方向....",酞菁铁_聚苯乙炔_吸波材料,化学/化学工程与技术,工学
2,农用运输车柴油机排放控制探讨,"介绍了农用柴油机排放的生成机理,探讨了控制农用柴油机排放的具体措施,并对今后控制农用柴油机排...",农用柴油机_排放_控制技术,农业工程,工学
3,点式安装玻璃幕墙在火灾中的破裂行为,"使用500,mm×500,mm正庚烷池火作为火源，通过全尺寸实验研究了1,200,mm×1,...",点式安装玻璃幕墙_火灾_破裂行为_全尺寸实验_有限元方法,动力工程及工程热物理,工学
4,火箭橇多路时序点火控制系统设计,"为了解决火箭橇试验中橇体长时间巡航运动的问题,采取了火箭发动机橇载多路时序点火控制的方法,分...",时序点火_远程控制_控制信号_时间精度_可靠性,兵器科学与技术,工学
...,...,...,...,...,...
3169500,基于功能梯度表面的微型热管换热性能研究,"通过将液固接触角沿轴向呈阶梯状分布的功能表面引入到三角形微型热管的一维稳态模型之中,分析了其...",微型热管_功能表面_毛细力_剪切力,电子科学与技术,工学
3169501,严控人畜共患病保障公共卫生安全,"近年来,在世界范围频频发生口蹄疫、疯牛病、禽流感等多种人畜共患病,其严重地威胁人类健康和畜...",人畜共患病_公共_生活水平提高_畜牧业发展_发病率增加_动物性食品_卫生事件_卫生安全_生活...,畜牧学/兽医学,农学
3169502,摇摆状态下竖直管内单相水阻力特性实验研究,"在3种不同摇摆周期(5s、10s、15s)、2种不同摇摆角度(10°、20°)的情况下,分别...",摇摆_单相流_阻力特性_摩擦阻力系数,核科学与技术,工学
3169503,私有出租房屋的社会主义改造--以苏州市为个案的考察,私有出租房屋的社会主义改造是整个社会主义改造工作的重要组成部分。苏州市的私租房改造于1956...,私有出租住房_社会主义改造_私租房改造,历史学,历史学


In [4]:
data["subject"].unique()

array(['农业工程', '化学/化学工程与技术', '动力工程及工程热物理', '兵器科学与技术', '机械工程', '纺织科学与工程',
       '大气科学', '园艺学', '哲学', '力学', '水产', '林学/林业工程', '艺术学', '历史学', '农林经济管理',
       '地理学', '工商管理', '体育学', '畜牧学/兽医学', '冶金工程', '环境科学与工程', '航空宇航科学与技术',
       '船舶与海洋工程', '材料科学与工程', '信息与通信工程', '图书馆、情报与档案管理', '军事学', '核科学与技术',
       '建筑学', '矿业工程', '食品科学与工程', '公共管理', '光学工程', '社会学', '交通运输工程', '理论经济学',
       '中国语言文学', '土木工程', '电气工程', '药学', '民族学', '计算机科学与技术', '作物学',
       '基础医学/临床医学', '新闻传播学', '应用经济学', '电子科学与技术', '测绘科学与技术',
       '地质学/地质资源与地质工程', '中医学/中药学', '法学', '水利工程', '数学', '心理学',
       '生物学/生物科学与工程', '政治学', '石油与天然气工程', '公共卫生与预防医学', '农业资源利用', '海洋科学',
       '教育学', '口腔医学', '物理学', '控制科学与工程', '天文学', '植物保护', '地球物理学', '工学',
       '农学', '管理学', '经济学', '理学'], dtype=object)

In [5]:
data["category"].unique()

array(['工学', '理学', '农学', '哲学', '艺术学', '历史学', '管理学', '教育学', '军事学', '法学',
       '经济学', '文学', '医学', nan], dtype=object)

In [6]:
es = Elasticsearch(
    hosts="http://127.0.0.1:9204",
)
dict(es.info())

{'name': 'node0',
 'cluster_name': 'docbase',
 'cluster_uuid': '63TQlJkjRvyY3NtujxmYuQ',
 'version': {'number': '8.6.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '180c9830da956993e59e2cd70eb32b5e383ea42c',
  'build_date': '2023-01-24T21:35:11.506992272Z',
  'build_snapshot': False,
  'lucene_version': '9.4.2',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [7]:
index = index.InvertedIndex(es)
index

Invert Index Instance 21f1cb1f4c0 <Elasticsearch(['http://127.0.0.1:9204'])> co_occ_test

In [8]:
try:
    res = index.delete_index_and_data()
    print(res)
except Exception as e:
    print(e)

NotFoundError(404, 'index_not_found_exception', 'no such index [co_occ_test]', co_occ_test, index_or_alias)


In [9]:
print(index.set_index())
index.index_data(data["content"])


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'co_occ_test'}


(3169505, [])

In [10]:
index

Invert Index Instance 21f1cb1f4c0 <Elasticsearch(['http://127.0.0.1:9204'])> co_occ_test